In [1]:
from __future__ import print_function, division
import time

from matplotlib import rcParams
import matplotlib.pyplot as plt

from nilmtk import DataSet, TimeFrame, MeterGroup
from ext_daedisaggregator import DAEDisaggregator
import ext_metrics as metrics

2023-02-01 01:25:48.469015: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
print("========== OPEN DATASETS ============")
train = DataSet('data/ukdale.h5')
test = DataSet('data/ukdale.h5')

========== OPEN DATASETS ============


In [3]:
train.set_window(start="13-4-2013", end="1-1-2014")
test.set_window(start="1-1-2014", end="30-3-2014")

In [4]:
from nilmtk.utils import print_dict
# print_dict(train.metadata)

train.buildings[1].elec

MeterGroup(meters=
  ElecMeter(instance=2, building=1, dataset='UK-DALE', appliances=[Appliance(type='boiler', instance=1)])
  ElecMeter(instance=3, building=1, dataset='UK-DALE', appliances=[Appliance(type='solar thermal pumping station', instance=1)])
  ElecMeter(instance=4, building=1, dataset='UK-DALE', appliances=[Appliance(type='laptop computer', instance=1), Appliance(type='laptop computer', instance=3)])
  ElecMeter(instance=5, building=1, dataset='UK-DALE', appliances=[Appliance(type='washer dryer', instance=1), Appliance(type='washer dryer', instance=2)])
  ElecMeter(instance=6, building=1, dataset='UK-DALE', appliances=[Appliance(type='dish washer', instance=1)])
  ElecMeter(instance=7, building=1, dataset='UK-DALE', appliances=[Appliance(type='television', instance=1)])
  ElecMeter(instance=8, building=1, dataset='UK-DALE', appliances=[Appliance(type='light', instance=1), Appliance(type='light', instance=2)])
  ElecMeter(instance=9, building=1, dataset='UK-DALE', appliances

In [5]:
train_building = 1
test_building = 1
sample_period = 6
meter_key = 'microwave'
train_elec = train.buildings[train_building].elec
test_elec = test.buildings[test_building].elec

train_meter = train_elec.submeters()[meter_key]
test_meter = test_elec.submeters()[meter_key]
train_mains = train_elec.mains()
test_mains = test_elec.mains()
dae = DAEDisaggregator(300)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


2023-02-01 01:25:58.767289: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
start = time.time()
print("========== TRAIN ============")
epochs = 0
for i in range(3):
    print("CHECKPOINT {}".format(epochs))
    dae.train(train_mains, train_meter, epochs=5, sample_period=sample_period)
    epochs += 5
    dae.export_model("UKDALE-DAE-h{}-{}-{}epochs.h5".format(train_building,
                                                        meter_key,
                                                        epochs))
end = time.time()
print("Train =", end-start, "seconds.")

========== TRAIN ============
CHECKPOINT 0
Epoch 1/5
790/790 [==============================] - 45s 55ms/step - loss: 1.2931e-04
Epoch 2/5
790/790 [==============================] - 45s 57ms/step - loss: 1.2901e-04
Epoch 3/5
790/790 [==============================] - 44s 56ms/step - loss: 1.2896e-04
Epoch 4/5
790/790 [==============================] - 45s 56ms/step - loss: 1.2902e-04
Epoch 5/5
790/790 [==============================] - 45s 57ms/step - loss: 1.2894e-04
CHECKPOINT 5
Epoch 1/5
790/790 [==============================] - 46s 58ms/step - loss: 1.2897e-04
Epoch 2/5
790/790 [==============================] - 47s 60ms/step - loss: 1.2893e-04
Epoch 3/5
790/790 [==============================] - 46s 59ms/step - loss: 1.2901e-04
Epoch 4/5
790/790 [==============================] - 52s 66ms/step - loss: 1.2903e-04
Epoch 5/5
790/790 [==============================] - 46s 58ms/step - loss: 1.2900e-04
CHECKPOINT 10
Epoch 1/5
790/790 [==============================] - 49s 62ms/step - l

In [8]:
from nilmtk import HDFDataStore
print("========== DISAGGREGATE ============")
disag_filename = "disag-out.h5"
output = HDFDataStore(disag_filename, 'w')
dae.disaggregate(test_mains, output, test_meter, sample_period=sample_period)
output.close()

========== DISAGGREGATE ============


AttributeError: type object 'object' has no attribute 'dtype'

In [ ]:

print("========== RESULTS ============")
result = DataSet(disag_filename)
res_elec = result.buildings[test_building].elec
rpaf = metrics.recall_precision_accuracy_f1(res_elec[meter_key], test_meter)
print("============ Recall: {}".format(rpaf[0]))
print("============ Precision: {}".format(rpaf[1]))
print("============ Accuracy: {}".format(rpaf[2]))
print("============ F1 Score: {}".format(rpaf[2]))

print("============ Relative error in total energy: {}".format(metrics.relative_error_total_energy(res_elec[meter_key], test_meter)))
print("============ Mean absolute error(in Watts): {}".format(metrics.mean_absolute_error(res_elec[meter_key], test_meter)))